In [46]:
# importing libraries
from datetime import datetime, timezone, timedelta, time
import pandas as pd
import numpy as np
import talib
import csv
import time as tm
import re
import json
from tqdm import tqdm
import logging
import timeit
import concurrent.futures


# Spot Data

---

In [2]:
# Reading Spot Data
df = pd.read_csv('NiftySpotData.csv')

In [3]:
print(df.tail(10))

       Ticker                   Datetime      Open      High       Low  \
465917  NIFTY  2023-01-31 15:23:00+05:30  17672.10  17672.85  17667.60   
465918  NIFTY  2023-01-31 15:24:00+05:30  17668.35  17674.15  17668.35   
465919  NIFTY  2023-01-31 15:25:00+05:30  17669.50  17670.10  17665.10   
465920  NIFTY  2023-01-31 15:26:00+05:30  17665.75  17673.15  17665.75   
465921  NIFTY  2023-01-31 15:27:00+05:30  17670.55  17678.00  17670.55   
465922  NIFTY  2023-01-31 15:28:00+05:30  17676.10  17677.90  17671.00   
465923  NIFTY  2023-01-31 15:29:00+05:30  17672.35  17682.90  17671.70   
465924  NIFTY  2023-01-31 15:30:00+05:30  17678.55  17686.70  17664.60   
465925  NIFTY  2023-01-31 15:31:00+05:30  17682.30  17682.30  17682.30   
465926  NIFTY  2023-01-31 15:32:00+05:30  17662.15  17662.15  17662.15   

           Close  
465917  17668.45  
465918  17670.25  
465919  17666.15  
465920  17670.20  
465921  17675.65  
465922  17672.60  
465923  17681.85  
465924  17682.55  
465925  17682.

In [4]:
# Converting 1 min canlde to 15 min candle

# Convert 'Datetime' column to datetime type if it's not already
df['Datetime'] = pd.to_datetime(df['Datetime'], format='%Y-%m-%d %H:%M:%S%z')

# Determine the start time of the first interval
start_time_first_interval = df['Datetime'].iloc[0].replace(hour=9, minute=30, second=0, microsecond=0)

# Calculate the number of 15-minute intervals from the market opening time (9:30 AM)
df['Interval'] = ((df['Datetime'] - start_time_first_interval) // timedelta(minutes=15)).astype(int)

# Group by the new 'Interval' column and aggregate open, high, low, close values
result_df = df.groupby('Interval').agg({
    'High': 'max',
    'Low': 'min',
    'Open': 'first',
    'Close': 'last'
})

# Reset the index
result_df.reset_index(inplace=True)

# Calculate the datetime using the start time of the first interval
result_df['Datetime'] = start_time_first_interval + result_df['Interval'] * timedelta(minutes=15)

# Drop the 'Interval' column if you don't need it in the final result
result_df.drop(columns='Interval', inplace=True)

result_df['Datetime'] = pd.to_datetime(result_df['Datetime'], format='%Y-%m-%d %H:%M:%S%z')

df = result_df


In [5]:
# Calculating the ADM (adapting moving average)
def adaptive_moving_average(close_prices, window=14, fast_factor=2.0, slow_factor=30.0):
    volatility = close_prices.pct_change().rolling(window=window, min_periods=window).std()
    fast_ema = close_prices.ewm(span=fast_factor * window, adjust=False).mean()
    slow_ema = close_prices.ewm(span=slow_factor * window, adjust=False).mean()
    ama = (fast_ema + volatility * (close_prices - slow_ema)).ewm(span=window, adjust=False).mean()
    return ama

df['ama'] = adaptive_moving_average(df['Close'], window=14, fast_factor=2, slow_factor=30)

In [6]:
# Calculating the RSI
df['rsi'] = talib.RSI(df['Close'], timeperiod=14)

In [7]:
# Calculate ATR
pd.set_option("display.max_rows", None, "display.max_columns", None)
np.set_printoptions(suppress=True,linewidth=np.nan)
pd.options.display.float_format = '{:,.4f}'.format


def calculate_atr(data, period=14):
    # Calculate True Range (TR)
    data['high-low'] = data['High'] - data['Low']
    data['high-close_prev'] = abs(data['High'] - data['Close'].shift(1))
    data['low-close_prev'] = abs(data['Low'] - data['Close'].shift(1))
    data['true_range'] = data[['high-low', 'high-close_prev', 'low-close_prev']].max(axis=1)

    # Calculate ATR
    data['atr'] = data['true_range'].rolling(window=period, min_periods=1).mean()

    # Drop intermediate columns
    data.drop(['high-low', 'high-close_prev', 'low-close_prev', 'true_range'], axis=1, inplace=True)

    return data


df = calculate_atr(df, period=14)


In [8]:
# Calculating Pivot Points
def calculate_pivot_points(data, column_name, length=14):
    if column_name == 'High':
        pivot_points = pd.DataFrame(index=data.index, columns=[column_name, f'IsPivot{column_name.capitalize()}'])

        for i in range(length, len(data) - length):
            # Check if the current point is a pivot high
            is_pivot_point = all(data[column_name][i] > data[column_name][i - length:i]) and all(data[column_name][i] > data[column_name][i + 1:i + length + 1])
            # is_pivot_point = all(data[column_name][i] > data[column_name][i - length:i]) 

            # Store the values in the DataFrame
            pivot_points.at[data.index[i], column_name] = data[column_name][i] if is_pivot_point else None

            pivot_points.at[data.index[i], f'IsPivot{column_name.capitalize()}'] = is_pivot_point
    
    elif column_name == 'Low':
        pivot_points = pd.DataFrame(index=data.index, columns=[column_name, f'IsPivot{column_name.capitalize()}'])
        for i in range(length, len(data) - length):
            # Check if the current point is a pivot low
            is_pivot_point = all(data[column_name][i] < data[column_name][i - length:i]) and all(data[column_name][i] < data[column_name][i + 1:i + length + 1])
            # is_pivot_point = all(data[column_name][i] < data[column_name][i - length:i])

            # Store the values in the DataFrame
            pivot_points.at[data.index[i], column_name] = data[column_name][i] if is_pivot_point else None

            pivot_points.at[data.index[i], f'IsPivot{column_name.capitalize()}'] = is_pivot_point

    return pivot_points


PH = calculate_pivot_points(df, 'High', length=14)
df['PL'] = calculate_pivot_points(df, 'Low', length=14)['IsPivotLow']

df['PH'] = PH['IsPivotHigh'] 


for i in range(1, len(df)):
    if df.at[df.index[i], 'PH']:
        df.at[df.index[i], 'PH_val'] = df.at[df.index[i], 'High']
    else:
        df.at[df.index[i], 'PH_val'] = df.at[df.index[i - 1], 'PH_val']
    if df.at[df.index[i], 'PL']:
        df.at[df.index[i], 'PL_val'] = df.at[df.index[i], 'Low']
    else:
        df.at[df.index[i], 'PL_val'] = df.at[df.index[i - 1], 'PL_val']

df['Slope'] = df['atr']/14

# Initialize the first values for slope_ph and slope_pl
df.at[df.index[0], 'slope_ph'] = df.at[df.index[0], 'Slope']
df.at[df.index[0], 'slope_pl'] = df.at[df.index[0], 'Slope']


# Update slope_ph and slope_pl based on conditions
for i in range(1, len(df)):
    if df.at[df.index[i], 'PH']:
        df.at[df.index[i], 'slope_ph'] = df.at[df.index[i], 'Slope']
    else:
        df.at[df.index[i], 'slope_ph'] = df.at[df.index[i - 1], 'slope_ph']
    if df.at[df.index[i], 'PL']:
        df.at[df.index[i], 'slope_pl'] = df.at[df.index[i], 'Slope']
    else:
        df.at[df.index[i], 'slope_pl'] = df.at[df.index[i - 1], 'slope_pl']



df['upper'] = 0
df['lower'] = 0
# Update upper and lower based on conditions

for i in range(1, len(df)):
    if df.at[df.index[i], 'PH'] == True:
        df.at[df.index[i], 'upper'] = df.at[df.index[i], 'High']
    else:
        df.at[df.index[i], 'upper'] = df.at[df.index[i - 1], 'upper'] - df.at[df.index[i], 'slope_ph']

for i in range(1, len(df)):
    if df.at[df.index[i], 'PL'] == True:
        df.at[df.index[i], 'lower'] = df.at[df.index[i], 'Low']
    else:
        df.at[df.index[i], 'lower'] = df.at[df.index[i - 1], 'lower'] + df.at[df.index[i], 'slope_pl']
        

# length = 14
df['upos'] = 0
df['dnos'] = 0

# Calculate upos
for i in range(1, len(df)):
    if df.at[df.index[i], 'PH'] != True or df.at[df.index[i], 'PL'] =='':
        upper_limit = df.at[df.index[i - 1], 'upper']
        if df.at[df.index[i], 'Close'] > upper_limit:
            df.at[df.index[i], 'upos'] = 1

# Calculate dnos
for i in range(1, len(df)):
    if df.at[df.index[i], 'PL'] != True or df.at[df.index[i], 'PH'] == '':
        lower_limit = df.at[df.index[i - 1], 'lower']
        if df.at[df.index[i], 'Close'] < lower_limit:
            df.at[df.index[i], 'dnos'] = 1



/var/folders/fv/h8flhzz90ljgbvrrq5wxfv4w0000gn/T/ipykernel_4620/3695917343.py:75: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.625' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[df.index[i], 'upper'] = df.at[df.index[i - 1], 'upper'] - df.at[df.index[i], 'slope_ph']
/var/folders/fv/h8flhzz90ljgbvrrq5wxfv4w0000gn/T/ipykernel_4620/3695917343.py:81: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.625' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[df.index[i], 'lower'] = df.at[df.index[i - 1], 'lower'] + df.at[df.index[i], 'slope_pl']


In [9]:
# Calulating the pivot highs and pivot lows
pivot = calculate_pivot_points(df, 'High', length=14)
df['IsPivotHigh'] = pivot['IsPivotHigh']

In [10]:
# Calculating the signals

df['signal'] = 'Hold'

buy_condition = (df['upos'] > df['upos'].shift(1)) & (df['signal'] != 'Buy')
sell_condition = (df['dnos'] > df['dnos'].shift(1)) & (df['signal'] != 'Sell')

df.loc[buy_condition, 'signal'] = 'Buy'
df.loc[sell_condition, 'signal'] = 'Sell'




In [11]:
# Saving the data SPOT to a csv file
df.to_csv('NiftySpotData1.csv')


# Options Data

---

In [2]:
# importing libraries
from datetime import datetime, timezone, timedelta, time
import pandas as pd
import numpy as np
import talib
import csv
import time as tm
import re
import json
from tqdm import tqdm
import logging
import timeit
import concurrent.futures


In [3]:
df = pd.read_csv('NiftySpotData1.csv')
df['Datetime'] = pd.to_datetime(df['Datetime'], format='%Y-%m-%d %H:%M:%S%z')

In [4]:
# Read the options data
optionsData = pd.read_csv('optionsDataDataframeReadyFromMerged2022.csv')

In [13]:
optionsData.head()


,Unnamed: 0.1,Unnamed: 0,Ticker,Open,High,Low,Close,Volume,Open Interest,Datetime,ExpiryDate
0,1907019,3663328,NIFTY06JAN2217450CE.NFO,84.0500,103.3000,78.9500,98.7000,380450,818400,2022-01-03 09:15:59,2022-01-06 00:00:00+00:00
1,1942578,3698887,NIFTY13JAN2218350CE.NFO,4.9500,4.9500,4.9000,4.9000,200,1550,2022-01-03 09:15:59,2022-01-13 00:00:00+00:00
2,1988085,3744394,NIFTY31MAR2218000CE.NFO,309.4500,309.4500,309.4500,309.4500,100,354800,2022-01-03 09:15:59,2022-03-31 00:00:00+00:00
3,1988659,3744968,NIFTY31MAR2218500CE.NFO,193.1500,193.1500,193.1500,193.1500,50,214450,2022-01-03 09:15:59,2022-03-31 00:00:00+00:00
4,1989019,3745328,NIFTY31MAR2219000CE.NFO,69.0500,69.0500,69.0000,69.0000,250,378250,2022-01-03 09:15:59,2022-03-31 00:00:00+00:00


In [ ]:
optionsData.describe()

In [14]:
# Converting Datetime to datetime type
optionsData['Datetime'] = pd.to_datetime(optionsData['Datetime'], format='%Y-%m-%d %H:%M:%S')


In [15]:
# drop column 'Unnamed: 0' and 'Unnamed: 0.1'
optionsData.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [16]:
# Function to get the closest expiry date from the optionsData.csv

#### old Function
# def get_closest_expiry_date(datetime_str, options_data):
#     # Parse the original datetime string
#     original_datetime = datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:%S%z")
    
#     try:
#         # Filter out dates in the past and keep only those within 7 days ahead of original_datetime
#         filtered_dates = options_data[
#             (options_data['ExpiryDate'] >= original_datetime) &
#             (options_data['ExpiryDate'] <= original_datetime + timedelta(days=7))
#         ]
#     except TypeError:
#         options_data['ExpiryDate'] = options_data['ExpiryDate'].dt.tz_localize(None)
#         # Filter out dates in the past and keep only those within 7 days ahead of original_datetime
#         filtered_dates = options_data[
#             (options_data['ExpiryDate'] >= original_datetime) &
#             (options_data['ExpiryDate'] <= original_datetime + timedelta(days=7))
#         ]
    
#     if filtered_dates.empty:
#         # print("No valid dates in the future.")
#         return None

#     # Sort the DataFrame by 'ExpiryDate'
#     filtered_dates = filtered_dates.sort_values(by='ExpiryDate')

#     # Calculate the absolute difference between the given datetime and each date in the filtered DataFrame
#     filtered_dates['Difference'] = abs(filtered_dates['ExpiryDate'] - original_datetime)

#     # Find the date with the minimum absolute difference among the filtered dates
#     closest_date = filtered_dates.iloc[0]['ExpiryDate']

#     return closest_date


#### Time improvement Function

def get_closest_expiry_date(datetime_str, options_data):
    # Parse the original datetime string
    original_datetime = datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:%S%z")

    # Filter out dates in the past and keep only those within 7 days ahead of original_datetime
    filtered_dates = options_data[
        (options_data['ExpiryDate'] >= original_datetime) &
        (options_data['ExpiryDate'] <= original_datetime + timedelta(days=7))
    ]

    if filtered_dates.empty:
        logging.info("No valid dates in the future.")
        return None

    # Find the date with the minimum absolute difference among the filtered dates
    closest_date = filtered_dates['ExpiryDate'].min()

    return closest_date



In [17]:
# Function to get the ticker name from optionData.csv

def get_ticker_name(symbol, datetime_str, atm):
    # convert Example 2018-01-27 09:30:00+05:30 to 27JAN18
    # Parse the original datetime string
    original_datetime = datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:%S%z")
    # Format the datetime as "27JAN18"
    formatted_datetime_str = original_datetime.strftime("%d%b%y").upper()


    tickerCE = f"{symbol.upper()}{formatted_datetime_str}{atm}CE.NFO"
    tickerPE = f"{symbol.upper()}{formatted_datetime_str}{atm}PE.NFO"

    is_present_ce = tickerCE in optionsData['Ticker'].values
    is_present_pe = tickerPE in optionsData['Ticker'].values
    
    if is_present_ce and is_present_pe:
        return tickerCE, tickerPE
    elif is_present_ce and not is_present_pe:
        return tickerCE, None
    elif is_present_pe and not is_present_ce:
        return None, tickerPE
    else:
        logging.warning(f"Neither {tickerCE} nor {tickerPE} is present in the options data. At {datetime_str} for ATM {atm}.")
        return None, None


In [18]:
# get close price for the given ticker NIFTY06JAN2217450CE.NFO from optionsData for the given datetime

def get_close_price(ticker, datetime_str, optionsData):
    # Parse the original datetime string
    original_datetime = datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:%S")

    # Subtract 1 second from the original datetime
    original_datetime -= timedelta(seconds=1)

    # Create boolean masks for filtering
    datetime_mask = optionsData['Datetime'] == original_datetime
    ticker_mask = optionsData['Ticker'] == ticker

    # Filter the DataFrame using boolean masks
    filtered_rows = optionsData[datetime_mask & ticker_mask]

    if not filtered_rows.empty:
        # Extract the close price from the filtered row
        return filtered_rows['Close'].iloc[0]
    else:
        logging.warning(f"No data found for {ticker} at {original_datetime}")
        return None
        


In [19]:
# Formating the ExpiryDate to the required format
try:
    optionsData['ExpiryDate'] = pd.to_datetime(optionsData['ExpiryDate'], format='%Y-%m-%d %H:%M:%S%z', errors='coerce')
except Exception as e:
    pass


In [20]:
# Check for the above functions


start_time = timeit.default_timer()
closest_date = get_closest_expiry_date('2022-12-04 09:30:00+05:30', optionsData)
end_time = timeit.default_timer()
print('Time taken by get_closest_epiry_date: ', end_time - start_time)

print(closest_date)

start_time = timeit.default_timer()
ce, pe = get_ticker_name('NIFTY', str(closest_date), 17050)
end_time = timeit.default_timer()
print('Time taken by get_ticker_name: ', end_time - start_time)
print(ce, pe)

start_time = timeit.default_timer()
print(get_close_price('NIFTY06JAN2217450CE.NFO', '2022-02-03 09:30:00', optionsData))
end_time = timeit.default_timer()
print('Time taken by get_close_price: ', end_time - start_time)



Time taken by get_closest_epiry_date:  0.2636056640003517
2022-12-08 00:00:00+00:00
Time taken by get_ticker_name:  1.0018759459999274
None NIFTY08DEC2217050PE.NFO


None
Time taken by get_close_price:  1.7725309040001775


In [21]:
def put_atmsp_wingput_wingcall_to_df(df, index, atmSP, wingCall, wingPut):
    try:
        # Store the values in the dataframe
        df.loc[df.index[index], ['atmSP', 'wingPut', 'wingCall']] = [atmSP, wingPut, wingCall]
        return df
    except Exception as e:
        logging.error(f"Error in put_atmsp_wingput_wingcall_to_df: {e}")


def get_close_price_dict(tickers: dict, optionsData: pd.DataFrame, datetime_str):
    try:
        # Define a list to store the futures
        futures = []

        # Create a ThreadPoolExecutor with a maximum of 4 threads
        with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
            # Submit tasks to fetch close prices asynchronously
            for ticker_name in tickers.values():
                futures.append(executor.submit(get_close_price, ticker_name, datetime_str, optionsData))

        # Retrieve the results
        results = [future.result() for future in futures]

        # Map the results to the ticker names
        close_prices = {
            'atmSPPut': results[0],
            'atmSPCall': results[1],
            'wingPutPrice': results[2],
            'wingCallPrice': results[3]
        }

        return close_prices
    except Exception as e:
        logging.error(f"Error in get_close_price_dict: {e}")





def get_tickers_dict(datetime_str, df, optionsData, atmSP, wingPut, wingCall):
    try:
        # Get the closest expiry date
        closest_date = str(get_closest_expiry_date(datetime_str, optionsData))

        # Define a list to store the futures
        futures = []

        # Create a ThreadPoolExecutor with a maximum of 4 threads
        with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
            # Submit tasks to fetch ticker names asynchronously
            futures.append(executor.submit(get_ticker_name, 'NIFTY', closest_date, atmSP))
            futures.append(executor.submit(get_ticker_name, 'NIFTY', closest_date, wingPut))
            futures.append(executor.submit(get_ticker_name, 'NIFTY', closest_date, wingCall))


        # Construct the tickers dictionary
        tickers_dict = {
            'atmce': futures[0].result()[0],
            'atmpe': futures[0].result()[1],
            'wingce': futures[1].result()[1],
            'wingpe': futures[2].result()[0]
        }

        return tickers_dict
    except Exception as e:
        logging.error(f"Error in get_tickers_dict: {e}")





In [22]:
# All rolling dictionary related functions

##### Fucntions to generate blank the rolling_dict

# Specify the number of legs
num_legs = 4

# Function to create a sub-dictionary with the specified number of legs
def create_legs_dict(prefix, num_legs):
    return {f'{prefix}{i}': 0 for i in range(1, num_legs + 1)}

# Function to create a current and previous data dictionary
def create_empty_data_dict(num_legs):
    return {
        'legPricesOrignal': create_legs_dict(prefix='legPriceOrignal', num_legs=num_legs),
        'position': 'buy',
        'balance': 0, 
        'lposs': create_legs_dict(prefix='lpos', num_legs=num_legs), #leg position
        'legsAfterPos': create_legs_dict(prefix='legAfterPos', num_legs=num_legs),
        'legsAfterPosDiff': create_legs_dict(prefix='legAfterPosDiff', num_legs=num_legs),
        'legsPricesFinal': create_legs_dict(prefix='legPriceFinal', num_legs=num_legs),
        'm2ms': create_legs_dict(prefix='m2m', num_legs=num_legs),
        'totalPL': 0,
        'cumReturns': 0
    }


# function to build the current data dictionary
def build_current_data_dict(price_dict, position):
    # Create a dictionary to store the current data
    current_data_dict = {
        'legPricesOrignal': {
        'legPriceOrignal1': price_dict['atmSPPut'],
        'legPriceOrignal2': price_dict['atmSPCall'],
        'legPriceOrignal3': price_dict['wingPutPrice'],
        'legPriceOrignal4': price_dict['wingCallPrice']},
        'position': position,
        'balance': 0,
        'lposs': create_legs_dict(prefix='lpos', num_legs=num_legs),
        'legsAfterPos': create_legs_dict(prefix='legAfterPos', num_legs=num_legs),
        'legsAfterPosDiff': create_legs_dict(prefix='legAfterPosDiff', num_legs=num_legs),
        'legsPricesFinal': create_legs_dict(prefix='legPriceFinal', num_legs=num_legs),
        'm2ms': create_legs_dict(prefix='m2m', num_legs=num_legs),
        'totalPL': 0,
        'cumReturns': 0
    }

    return current_data_dict

def update_rolling_dict(rolling_dict, current_data: dict):
    # Update the 'previous_data' dictionary in the 'rolling_dict'
    rolling_dict['previous_data'] = rolling_dict['current_data']

    # Update the 'current_data' dictionary in the 'rolling_dict'
    rolling_dict['current_data'] = current_data

    return rolling_dict



# Create dictionaries
previous_data_dict = create_empty_data_dict(num_legs=num_legs)
current_data_dict = create_empty_data_dict(num_legs=num_legs)

# Combine both dictionaries into a single dictionary
rolling_dict = {
    'previous_data': previous_data_dict,
    'current_data': current_data_dict
}





# json_str = json.dumps(rolling_dict, indent=4)
# print(json_str)

In [23]:
positions = {'beginx': [-1, -1, 1, 1], 'buy': [-2, 0, 2, 2], 'sell': [0, -2, 2, 2], 'squareoff': [0, 0, 0, 0]}


In [45]:


def write_current_data_to_dataframe(rolling_dict, df, df_index):
    # Write the 'current_data' dictionary to the DataFrame
    for key, value in rolling_dict['current_data'].items():
        if isinstance(value, dict):
            for sub_key, sub_value in value.items():
                df.at[df_index, f"{sub_key}"] = sub_value
        else:
            df.at[df_index, key] = value

    return df



def check_stoploss(rolling_dict, capital_at_start_of_trade):
    # Calculate the difference in capital from the start to the current balance
    difference_in_capital = capital_at_start_of_trade - rolling_dict['current_data']['balance']
    
    # Check if the difference is more than 3% of the initial capital
    return difference_in_capital > 0.03 * capital_at_start_of_trade






# fucntion to calculate the pnl
def calculate_m2m(rolling_dict, positions, zero=False):
    position_to_take = rolling_dict['current_data']['position']

    if zero:
        # zero out the current data
        rolling_dict['current_data'] = {key: (0 if not isinstance(value, dict) else {k: 0 for k in value}) for key, value in rolling_dict['current_data'].items()} 
        # carry forward the previous data
        rolling_dict['current_data'] = rolling_dict['previous_data']
    else:
        for i in range(0, num_legs):
            # Updating the postion taken
            rolling_dict['current_data']['lposs'][f"lpos{i+1}"] = positions[position_to_take][i]

        for i in range(0, num_legs):
            # Calculating the leg after position
            rolling_dict['current_data']['legsAfterPos'][f"legAfterPos{i+1}"] = rolling_dict['current_data']['legPricesOrignal'][f"legPriceOrignal{i+1}"] * rolling_dict['current_data']['lposs'][f"lpos{i+1}"]

            # Calculating the difference between the leg after position and the previous leg after position
            difference_in_position = rolling_dict['current_data']['lposs'][f"lpos{i+1}"] - rolling_dict['previous_data']['lposs'][f"lpos{i+1}"]
            rolling_dict['current_data']['legsAfterPosDiff'][f"legAfterPosDiff{i+1}"] = difference_in_position * rolling_dict['current_data']['legPricesOrignal'][f"legPriceOrignal{i+1}"]

            # Calculating Final Leg Price
            rolling_dict['current_data']['legsPricesFinal'][f"legPriceFinal{i+1}"] = rolling_dict['current_data']['legsAfterPosDiff'][f"legAfterPosDiff{i+1}"] + rolling_dict['previous_data']['legsAfterPos'][f"legAfterPos{i+1}"]

            # Calculating m2m
            rolling_dict['current_data']['m2ms'][f"m2m{i+1}"] = rolling_dict['current_data']['legsPricesFinal'][f"legPriceFinal{i+1}"] - rolling_dict['current_data']['legsAfterPos'][f"legAfterPos{i+1}"]

            
            # calculating the current data pnl which is the sum of all m2ms
            rolling_dict['current_data']['totalPL'] += rolling_dict['current_data']['m2ms'][f"m2m{i+1}"]

        # multiplying the totalPL by 50 for the lot size
        rolling_dict['current_data']['totalPL'] = rolling_dict['current_data']['totalPL'] * -1 * 50
        rolling_dict['current_data']['cumReturns'] = rolling_dict['previous_data']['cumReturns'] + rolling_dict['current_data']['totalPL'] 
        # updating the balance
        rolling_dict['current_data']['balance'] = rolling_dict['previous_data']['balance'] + rolling_dict['current_data']['totalPL']


    return rolling_dict




def calculate_m2m1(rolling_dict, positions, zero=False):
    position_to_take = rolling_dict['current_data']['position']
    num_legs = 4

    if zero:
        rolling_dict['current_data'] = {key: (0 if not isinstance(value, dict) else {k: 0 for k in value}) for key, value in rolling_dict['current_data'].items()} 
        rolling_dict['current_data'].update(rolling_dict['previous_data'])  
    else:
        total_pl = 0

        def calculate_leg(i):
            leg_price_original = rolling_dict['current_data']['legPricesOrignal'][f"legPriceOrignal{i+1}"]
            lpos_current = rolling_dict['current_data']['lposs'][f"lpos{i+1}"]
            lpos_previous = rolling_dict['previous_data']['lposs'][f"lpos{i+1}"]

            rolling_dict['current_data']['legsAfterPos'][f"legAfterPos{i+1}"] = leg_price_original * lpos_current

            difference_in_position = lpos_current - lpos_previous
            rolling_dict['current_data']['legsAfterPosDiff'][f"legAfterPosDiff{i+1}"] = difference_in_position * leg_price_original

            final_leg_price = rolling_dict['current_data']['legsAfterPosDiff'][f"legAfterPosDiff{i+1}"] + rolling_dict['previous_data']['legsAfterPos'][f"legAfterPos{i+1}"]
            rolling_dict['current_data']['legsPricesFinal'][f"legPriceFinal{i+1}"] = final_leg_price

            m2m = final_leg_price - rolling_dict['current_data']['legsAfterPos'][f"legAfterPos{i+1}"]
            rolling_dict['current_data']['m2ms'][f"m2m{i+1}"] = m2m

            nonlocal total_pl
            total_pl += m2m

        with concurrent.futures.ThreadPoolExecutor() as executor:
            executor.map(calculate_leg, range(num_legs))

        # Update the current_data with previous_data
        for key in rolling_dict['previous_data']:
            if key not in rolling_dict['current_data']:
                rolling_dict['current_data'][key] = rolling_dict['previous_data'][key]

        rolling_dict['current_data']['totalPL'] = -1 * total_pl * 50
        rolling_dict['current_data']['cumReturns'] = rolling_dict['previous_data']['cumReturns'] + rolling_dict['current_data']['totalPL']
        rolling_dict['current_data']['balance'] = rolling_dict['previous_data']['balance'] + rolling_dict['current_data']['totalPL']

        print(rolling_dict)
    return rolling_dict


    

In [25]:
# Cropping SPOT data to specific date

# Convert 'Datetime' column to datetime type
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Filter data after '2022-01-01' and before a specific end date (replace 'your_end_date' with the desired end date)
start_date = '2022-01-03'
end_date = '2022-01-05'  # Replace with your desired end date

# Filtering the DataFrame
df = df[(df['Datetime'] > start_date) & (df['Datetime'] <= end_date)]
df = df.reset_index(drop=True)


In [26]:
daily_diff = 500


# Define the round_to_nearest_50 function
round_to_nearest_50 = np.vectorize(lambda x: round(x / 50) * 50)

# Apply the function to the entire 'Close' column
df['atmSP'] = round_to_nearest_50(df['Close'].values)





def get_daily_diff(date_string):
    date_object = datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S%z')
    day_of_week = date_object.weekday()
    # Define a dictionary to map each day to its corresponding difference
    day_diff_mapping = {
        "Monday": 400,
        "Tuesday": 300,
        "Wednesday": 200,
        "Thursday": 100,
        "Friday": 500,
        "Saturday": None,
        "Sunday": None
    }
    # Return the difference based on the day of the week
    return day_diff_mapping[datetime.strftime(date_object, '%A')]


In [27]:
# Function to determine if the current time is before the target time
def is_before_target_time(date_string, target_time="15:10:00+05:30"):
    # Convert the input string to a datetime object
    date_object = datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S%z')

    # Extract the time part from the date object
    time_part = date_object.time()

    # Extract the time part from the target time string
    target_time_part = datetime.strptime(target_time, '%H:%M:%S%z').time()

    # Compare the time parts
    return time_part < target_time_part


In [29]:
# function to save data to csv
def save_data():
    df.to_csv('testData1.csv')


In [47]:
logging.basicConfig(filename='logs.log', filemode='w', encoding='utf-8', level=logging.DEBUG)


In [48]:
position = 999

# set the initial capital and the current capital
initial_capital = capital = 100000

logging.info("*"*20)
logging.info(f"Initial Capital: {initial_capital}")


first_run = True
for i in tqdm(range(1, len(df))):
    # Check if this a new day and time is 09:30:00+05:30

    current_datetime_str = str(df.at[df.index[i], 'Datetime'])
    current_datetime_str_short = current_datetime_str[:-6]
    logging.info(f"Current Datetime: {current_datetime_str}")



    if '09:30:00+05:30' in current_datetime_str or first_run:
       
        # Calculate the ATM strike price
        atmSP = round_to_nearest_50(df.at[df.index[i], 'Close'])
        wingPut = atmSP - get_daily_diff(current_datetime_str)
        wingCall = atmSP + get_daily_diff(current_datetime_str)
        

        # marking the postion as 0
        position = 0
        df.at[df.index[i], 'position'] = position

         
        # store the values in the dataframe
        df = put_atmsp_wingput_wingcall_to_df(df, i, atmSP, wingPut, wingCall)

        start = timeit.default_timer()
        tickers_dict = get_tickers_dict(current_datetime_str, df, optionsData, atmSP, wingPut, wingCall)
        end = timeit.default_timer()
        logging.info(f"Time taken for get_tickers_dict: {end - start}")
        close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)
        
        rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, 'beginx'))
        
        
        if first_run: rolling_dict['previous_data']['balance'] = capital

        # start timing the execution for 9:30
        start_time_930 = timeit.default_timer()
        rolling_dict = calculate_m2m(rolling_dict, positions, first_run)
        
        df = write_current_data_to_dataframe(rolling_dict, df, i)


        first_run = False

        # sell atmSP call -> api call
        # sell atmSp put -> api call
        # buy wingPut -> api call
        # buy wingCall -> api call
        


    # check if time is before 15:15:00+05:30 and after 09:30:00+05:30
    elif is_before_target_time(current_datetime_str) and current_datetime_str != '15:15:30+05:30' and not is_before_target_time(current_datetime_str, target_time="09:30:00+05:30"):
        # this will get triggered for every candle in between of 9:30 and 15:15 except them
        # checking for stoploss
        if check_stoploss(rolling_dict, capital):
            df.at[df.index[i], 'stoploss'] = 'SL Triggered'
            logging.info(f"Stoploss Triggered at {current_datetime_str}")
            capital = rolling_dict['current_data']['balance']
            # squareoff all positions
            # marking the postion as 0
            position = 999
            df.at[df.index[i], 'position'] = position
            atmSP = round_to_nearest_50(df.at[df.index[i], 'Close'])
            wingPut = atmSP - get_daily_diff(current_datetime_str)
            wingCall = atmSP + get_daily_diff(current_datetime_str)

            # store the values in the dataframe
            df = put_atmsp_wingput_wingcall_to_df(df, i, atmSP, wingPut, wingCall)
            start_timer = timeit.default_timer()
            tickers_dict = get_tickers_dict(current_datetime_str, df, optionsData, atmSP, wingPut, wingCall)
            end_timer = timeit.default_timer()
            logging.info(f"Time taken for get_tickers_dict: {end_timer - start_timer}")
            start_timer = timeit.default_timer()
            close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)
            end_timer = timeit.default_timer()
            logging.info(f"Time taken for get_close_price_dict: {end_timer - start_timer}")
        
            rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, 'squareoff'))

            capital = rolling_dict['previous_data']['balance']

            rolling_dict = calculate_m2m(rolling_dict, positions)
            
            df = write_current_data_to_dataframe(rolling_dict, df, i)
        else:
            df.at[df.index[i], 'position'] = position
            # store the values in the dataframe
            df = put_atmsp_wingput_wingcall_to_df(df, i, atmSP, wingPut, wingCall)

            start_timer = timeit.default_timer()
            close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)
            end_timer = timeit.default_timer()
            logging.info(f"Time taken for get_close_price_dict: {end_timer - start_timer}")
            rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, rolling_dict['current_data']['position']))
            start_timer = timeit.default_timer()
            rolling_dict = calculate_m2m(rolling_dict, positions)
            end_timer = timeit.default_timer()
            logging.info(f"Time taken for calculate_m2m: {end_timer - start_timer}")
            
            df = write_current_data_to_dataframe(rolling_dict, df, i)


    
    # check if time is 15:15:00+05:30
    elif df.at[df.index[i], 'Datetime'].time() == time(15, 15) or '15:15:00+05:30' in current_datetime_str:
        #squareoff all positions
        
        # store the values in the dataframe
        df = put_atmsp_wingput_wingcall_to_df(df, i, atmSP, wingPut, wingCall)
        close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)
        rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, 'squareoff'))
        rolling_dict = calculate_m2m(rolling_dict, positions)
        df = write_current_data_to_dataframe(rolling_dict, df, i)

        # buy atmSp call -> api call
        # buy atmSp put -> api call
        # sell wingPut -> api call
        # sell wingCall -> api call
        
        df.at[df.index[i], 'position'] = 999
        
    else:
        # this is for time between 15:15:00+05:30 and 09:30:00+05:30
        # store the values in the dataframe
        df = put_atmsp_wingput_wingcall_to_df(df, i, atmSP, wingPut, wingCall)
        df.at[df.index[i], 'position'] = position
        close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)
        
        rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, rolling_dict['current_data']['position']))
        
        rolling_dict = calculate_m2m(rolling_dict, positions, zero=True)
        df= write_current_data_to_dataframe(rolling_dict, df, i)




    if position == 0:
        if df.at[df.index[i], 'signal'] == 'Buy':
            if df.at[df.index[i], 'rsi'] < 70 and df.at[df.index[i], 'ama'] < df.at[df.index[i], 'Close']:
                capital = rolling_dict['current_data']['balance']
                close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)
                rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, 'buy'))
                rolling_dict = calculate_m2m(rolling_dict, positions)
                df = write_current_data_to_dataframe(rolling_dict, df, i)
                # buy atmSp call -> api call
                # Sell atmSp put -> api call
                # buy wingCall -> api call
                # buy wingPut -> api call
                
                position = 1
        
        elif df.at[df.index[i], 'signal'] == 'Sell':
            if df.at[df.index[i], 'rsi'] > 30 and df.at[df.index[i], 'ama'] > df.at[df.index[i], 'Close']:
                # tickers = get_tickers_dict(df.at[df.index[i], 'Datetime'], df, optionsData, atmSP, wingPut, wingCall)
                capital = rolling_dict['current_data']['balance']
                close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)
                rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, 'sell'))
                rolling_dict = calculate_m2m(rolling_dict, positions)
                df = write_current_data_to_dataframe(rolling_dict, df, i)
                # sell atmSp call -> api call
                # buy atmSp put -> api call
                # Buy wingPut -> api call
                # Buy wingCall -> api call

                position = -1

    elif position == 1:
        if df.at[df.index[i], 'signal'] == 'Hold':
            if df.at[df.index[i], 'rsi'] > 70:
                capital = rolling_dict['current_data']['balance']
                tickers = get_tickers_dict(df.at[df.index[i], 'Datetime'], df, optionsData, atmSP, wingPut, wingCall)
                close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)
                rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, 'squareoff'))
                rolling_dict = calculate_m2m(rolling_dict, positions)
                df = write_current_data_to_dataframe(rolling_dict, df, i)
                # buy atmSp put -> api call x 2
                # sell wingPut -> api call x 2
                # sell wingCall -> api call x 2


                # Update ticker
                tickers = get_tickers_dict(df.at[df.index[i], 'Datetime'], df, optionsData, atmSP, wingPut, wingCall)

                position = 999
        
        elif df.at[df.index[i], 'signal'] == 'Sell':
            capital = rolling_dict['current_data']['balance']
            close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)
            # buy atmSp put -> api call
            # sell wingPut -> api call
            # sell wingCall -> api call
            rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, 'sqaureoff'))
            rolling_dict = calculate_m2m(rolling_dict, positions)
            df = write_current_data_to_dataframe(rolling_dict, df, i)
            position = 999

    elif position == -1:
        if df.at[df.index[i], 'signal'] == 'Hold':
            if df.at[df.index[i], 'rsi'] < 30:
                tickers = get_tickers_dict(df.at[df.index[i], 'Datetime'], df, optionsData, atmSP, wingPut, wingCall)
                close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)
                # buy atmSp call -> api call
                # sell wingPut -> api call
                # sell wingCall -> api call
                rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, 'squareoff'))
                rolling_dict = calculate_m2m(rolling_dict, positions)
                df = write_current_data_to_dataframe(rolling_dict, df, i)
                # update tickers
                

                

                position = 999
                
        elif df.at[df.index[i], 'signal'] == 'Buy':
            if df.at[df.index[i], 'rsi'] < 30:
                capital = rolling_dict['current_data']['balance']
                close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)
                # buy atmSp call -> api call
                # sell wingPut -> api call
                # sell wingCall -> api call
                rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, 'squareoff'))
                rolling_dict = calculate_m2m(rolling_dict, positions)
                df = write_current_data_to_dataframe(rolling_dict, df, i)

                position = 999

    elif position == 999:
        if is_before_target_time(current_datetime_str):
            atmSP = round_to_nearest_50(df.at[df.index[i], 'Close'])
            wingPut = atmSP - get_daily_diff(current_datetime_str)
            wingCall = atmSP + get_daily_diff(current_datetime_str)

            # store the values in the dataframe
            df = put_atmsp_wingput_wingcall_to_df(df, i, atmSP, wingPut, wingCall)
            
            position = 0
            df.at[df.index[i], 'position'] = position



        else:
            #squareoff all positions
            close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)

            rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, 'squareoff'))
            rolling_dict = calculate_m2m(rolling_dict, positions)
            df = write_current_data_to_dataframe(rolling_dict, df, i)

            position = 999
            df.at[df.index[i], 'position'] = position

    else:
        logging.warning(f"Last Else Block is hit! {current_datetime_str} {position} {df.at[df.index[i], 'signal']}")



    # df.at[df.index[i], 'position'] = position


save_data()

# for i in df.index:
#   print(df[i])





  0%|          | 0/53 [00:00<?, ?it/s]

  9%|▉         | 5/53 [00:40<06:24,  8.02s/it]


KeyboardInterrupt: 

In [32]:
# import concurrent.futures
# import logging

# position = 999

# # set the initial capital and the current capital
# initial_capital = capital = 100000

# logging.info(f"Initial Capital: {initial_capital}")


# first_run = True
# def process_dataframe_row(i, df):
        
#     for i in tqdm(range(1, len(df))):
#         # Check if this a new day and time is 09:30:00+05:30

#         current_datetime_str = str(df.at[df.index[i], 'Datetime'])
#         current_datetime_str_short = current_datetime_str[:-6]
#         logging.info(f"Current Datetime: {current_datetime_str}")



#         if '09:30:00+05:30' in current_datetime_str or first_run:
        
#             # Calculate the ATM strike price
#             atmSP = round_to_nearest_50(df.at[df.index[i], 'Close'])
#             wingPut = atmSP - get_daily_diff(current_datetime_str)
#             wingCall = atmSP + get_daily_diff(current_datetime_str)
            

#             # marking the postion as 0
#             position = 0
#             df.at[df.index[i], 'position'] = position

            
#             # store the values in the dataframe
#             df = put_atmsp_wingput_wingcall_to_df(df, i, atmSP, wingPut, wingCall)

#             start = timeit.default_timer()
#             tickers_dict = get_tickers_dict(current_datetime_str, df, optionsData, atmSP, wingPut, wingCall)
#             end = timeit.default_timer()
#             logging.info(f"Time taken for get_tickers_dict: {end - start}")
#             close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)
            
#             rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, 'beginx'))
            
            
#             if first_run: rolling_dict['previous_data']['balance'] = capital

#             # start timing the execution for 9:30
#             start_time_930 = timeit.default_timer()
#             rolling_dict = calculate_m2m(rolling_dict, positions, first_run)
            
#             df = write_current_data_to_dataframe(rolling_dict, df, i)


#             first_run = False

#             # sell atmSP call -> api call
#             # sell atmSp put -> api call
#             # buy wingPut -> api call
#             # buy wingCall -> api call
            


#         # check if time is before 15:15:00+05:30 and after 09:30:00+05:30
#         elif is_before_target_time(current_datetime_str) and current_datetime_str != '15:15:30+05:30' and not is_before_target_time(current_datetime_str, target_time="09:30:00+05:30"):
#             # this will get triggered for every candle in between of 9:30 and 15:15 except them
#             # checking for stoploss
#             if check_stoploss(rolling_dict, capital):
#                 df.at[df.index[i], 'stoploss'] = 'SL Triggered'
#                 logging.info(f"Stoploss Triggered at {current_datetime_str}")
#                 capital = rolling_dict['current_data']['balance']
#                 # squareoff all positions
#                 # marking the postion as 0
#                 position = 999
#                 df.at[df.index[i], 'position'] = position
#                 atmSP = round_to_nearest_50(df.at[df.index[i], 'Close'])
#                 wingPut = atmSP - get_daily_diff(current_datetime_str)
#                 wingCall = atmSP + get_daily_diff(current_datetime_str)

#                 # store the values in the dataframe
#                 df = put_atmsp_wingput_wingcall_to_df(df, i, atmSP, wingPut, wingCall)
#                 tickers_dict = get_tickers_dict(current_datetime_str, df, optionsData, atmSP, wingPut, wingCall)
#                 close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)
            
#                 rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, 'squareoff'))

#                 capital = rolling_dict['previous_data']['balance']

#                 rolling_dict = calculate_m2m(rolling_dict, positions)
                
#                 df = write_current_data_to_dataframe(rolling_dict, df, i)
#             else:
#                 df.at[df.index[i], 'position'] = position
#                 # store the values in the dataframe
#                 df = put_atmsp_wingput_wingcall_to_df(df, i, atmSP, wingPut, wingCall)

#                 close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)
#                 rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, rolling_dict['current_data']['position']))
                
#                 rolling_dict = calculate_m2m(rolling_dict, positions)
                
#                 df = write_current_data_to_dataframe(rolling_dict, df, i)


        
#         # check if time is 15:15:00+05:30
#         elif df.at[df.index[i], 'Datetime'].time() == time(15, 15) or '15:15:00+05:30' in current_datetime_str:
#             #squareoff all positions
            
#             # store the values in the dataframe
#             df = put_atmsp_wingput_wingcall_to_df(df, i, atmSP, wingPut, wingCall)
#             close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)
#             rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, 'squareoff'))
#             rolling_dict = calculate_m2m(rolling_dict, positions)
#             df = write_current_data_to_dataframe(rolling_dict, df, i)

#             # buy atmSp call -> api call
#             # buy atmSp put -> api call
#             # sell wingPut -> api call
#             # sell wingCall -> api call
            
#             df.at[df.index[i], 'position'] = 999
            
#         else:
#             # this is for time between 15:15:00+05:30 and 09:30:00+05:30
#             # store the values in the dataframe
#             df = put_atmsp_wingput_wingcall_to_df(df, i, atmSP, wingPut, wingCall)
#             df.at[df.index[i], 'position'] = position
#             close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)
            
#             rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, rolling_dict['current_data']['position']))
            
#             rolling_dict = calculate_m2m(rolling_dict, positions, zero=True)
#             df= write_current_data_to_dataframe(rolling_dict, df, i)




#         if position == 0:
#             if df.at[df.index[i], 'signal'] == 'Buy':
#                 if df.at[df.index[i], 'rsi'] < 70 and df.at[df.index[i], 'ama'] < df.at[df.index[i], 'Close']:
#                     capital = rolling_dict['current_data']['balance']
#                     close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)
#                     rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, 'buy'))
#                     rolling_dict = calculate_m2m(rolling_dict, positions)
#                     df = write_current_data_to_dataframe(rolling_dict, df, i)
#                     # buy atmSp call -> api call
#                     # Sell atmSp put -> api call
#                     # buy wingCall -> api call
#                     # buy wingPut -> api call
                    
#                     position = 1
            
#             elif df.at[df.index[i], 'signal'] == 'Sell':
#                 if df.at[df.index[i], 'rsi'] > 30 and df.at[df.index[i], 'ama'] > df.at[df.index[i], 'Close']:
#                     # tickers = get_tickers_dict(df.at[df.index[i], 'Datetime'], df, optionsData, atmSP, wingPut, wingCall)
#                     capital = rolling_dict['current_data']['balance']
#                     close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)
#                     rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, 'sell'))
#                     rolling_dict = calculate_m2m(rolling_dict, positions)
#                     df = write_current_data_to_dataframe(rolling_dict, df, i)
#                     # sell atmSp call -> api call
#                     # buy atmSp put -> api call
#                     # Buy wingPut -> api call
#                     # Buy wingCall -> api call

#                     position = -1

#         elif position == 1:
#             if df.at[df.index[i], 'signal'] == 'Hold':
#                 if df.at[df.index[i], 'rsi'] > 70:
#                     capital = rolling_dict['current_data']['balance']
#                     tickers = get_tickers_dict(df.at[df.index[i], 'Datetime'], df, optionsData, atmSP, wingPut, wingCall)
#                     close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)
#                     rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, 'squareoff'))
#                     rolling_dict = calculate_m2m(rolling_dict, positions)
#                     df = write_current_data_to_dataframe(rolling_dict, df, i)
#                     # buy atmSp put -> api call x 2
#                     # sell wingPut -> api call x 2
#                     # sell wingCall -> api call x 2


#                     # Update ticker
#                     tickers = get_tickers_dict(df.at[df.index[i], 'Datetime'], df, optionsData, atmSP, wingPut, wingCall)

#                     position = 999
            
#             elif df.at[df.index[i], 'signal'] == 'Sell':
#                 capital = rolling_dict['current_data']['balance']
#                 close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)
#                 # buy atmSp put -> api call
#                 # sell wingPut -> api call
#                 # sell wingCall -> api call
#                 rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, 'sqaureoff'))
#                 rolling_dict = calculate_m2m(rolling_dict, positions)
#                 df = write_current_data_to_dataframe(rolling_dict, df, i)
#                 position = 999

#         elif position == -1:
#             if df.at[df.index[i], 'signal'] == 'Hold':
#                 if df.at[df.index[i], 'rsi'] < 30:
#                     tickers = get_tickers_dict(df.at[df.index[i], 'Datetime'], df, optionsData, atmSP, wingPut, wingCall)
#                     close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)
#                     # buy atmSp call -> api call
#                     # sell wingPut -> api call
#                     # sell wingCall -> api call
#                     rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, 'squareoff'))
#                     rolling_dict = calculate_m2m(rolling_dict, positions)
#                     df = write_current_data_to_dataframe(rolling_dict, df, i)
#                     # update tickers
                    

                    

#                     position = 999
                    
#             elif df.at[df.index[i], 'signal'] == 'Buy':
#                 if df.at[df.index[i], 'rsi'] < 30:
#                     capital = rolling_dict['current_data']['balance']
#                     close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)
#                     # buy atmSp call -> api call
#                     # sell wingPut -> api call
#                     # sell wingCall -> api call
#                     rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, 'squareoff'))
#                     rolling_dict = calculate_m2m(rolling_dict, positions)
#                     df = write_current_data_to_dataframe(rolling_dict, df, i)

#                     position = 999

#         elif position == 999:
#             if is_before_target_time(current_datetime_str):
#                 atmSP = round_to_nearest_50(df.at[df.index[i], 'Close'])
#                 wingPut = atmSP - get_daily_diff(current_datetime_str)
#                 wingCall = atmSP + get_daily_diff(current_datetime_str)

#                 # store the values in the dataframe
#                 df = put_atmsp_wingput_wingcall_to_df(df, i, atmSP, wingPut, wingCall)
                
#                 position = 0
#                 df.at[df.index[i], 'position'] = position



#             else:
#                 #squareoff all positions
#                 close_price_dict = get_close_price_dict(tickers_dict, optionsData, current_datetime_str_short)

#                 rolling_dict = update_rolling_dict(rolling_dict, build_current_data_dict(close_price_dict, 'squareoff'))
#                 rolling_dict = calculate_m2m(rolling_dict, positions)
#                 df = write_current_data_to_dataframe(rolling_dict, df, i)

#                 position = 999
#                 df.at[df.index[i], 'position'] = position

#         else:
#             logging.warning(f"Last Else Block is hit! {current_datetime_str} {position} {df.at[df.index[i], 'signal']}")



#         # df.at[df.index[i], 'position'] = position


#     save_data()

# # for i in df.index:
# #   print(df[i])



# with concurrent.futures.ThreadPoolExecutor() as executor:
#     # Submit tasks to the ThreadPoolExecutor for each row in the DataFrame
#     # Adjust the range as needed
#     futures = [executor.submit(process_dataframe_row, i, df) for i in range(1, len(df))]

#     # Iterate over the results if needed
#     for future in concurrent.futures.as_completed(futures):
#         try:
#             result = future.result()
#             # Process the result if needed
#         except Exception as exc:
#             # Handle exceptions raised during execution
#             logging.error(f"Error processing row: {exc}")


---
